# Particle database

:::{warning}

The {doc}`pycompwa </index>` is no longer maintained. Use the [ComPWA](https://compwa-org.rtfd.io) packages [QRules](https://qrules.rtfd.io), [AmpForm](https://ampform.rtfd.io), and [TensorWaves](https://tensorwaves.rtfd.io) instead!

:::

This notebook explains the difference between the particle list in {mod}`pycompwa` and the list used in the ComPWA C++ back-end. The two are only related the way they import and export XML files.

## pycompwa particle list

The {mod}`.particle` module is required for the expert system (see {doc}`/usage/workflow/step1`): when you create an instance of the {class}`.StateTransitionManager`, it loads the [default list of particles that is shipped with ComPWA](https://github.com/ComPWA/ComPWA/blob/master/Physics/particle_list.xml). This default list is loaded into a {class}`dict` called `particle_list` and is **used internally by the expert system**.

In [ ]:
from pycompwa.expertsystem.state.particle import particle_list
from pycompwa.expertsystem.ui.system_control import StateTransitionManager

initial_state = [("J/psi", [-1, 1])]
final_state = [("gamma"), ("pi0"), ("pi0")]
tbd_manager = StateTransitionManager(
    initial_state,
    final_state,
    formalism_type="helicity",
    topology_building="isobar",
)
original_number_of_entries = len(particle_list)
print("loaded", original_number_of_entries, "particles")

In PWA research, it is quite likely that this particle list is incomplete, particularly if you search for rare resonances or if you analyze a $e^+e^-$ collision of a particular energy as the initial state. You may therefore want to manipulate this particle list before you do call {meth}`.StateTransitionManager.find_solutions`. There are two ways to do this: (1) through Python or (2) by modifying a copy of the default XML file.

### (1) through Python

The particle list instance is a tree of {class}`~collections.OrderedDict` and {class}`dict` instances that represent the structure of the XML file.

In [ ]:
particle_list["D*(2010)+"]

Although this is a bit arduous to work with, it is possible to directly manipulate the entries in there through Python. Whatever modifications to the `particle_list` instance are later on also used in the expert system. In this case, for instance, we can set a wider width for the $D^*$:

In [ ]:
particle_list["D*(2010)+"]["DecayInfo"]["Parameter"][0]["Value"] = 0.01

It can also happen that you want to add new particles. Because of the hierarchical structure of entries in the `particle_list`, it is best to work with a {func}`~copy.deepcopy` of a similar particle that is already in that list. You can get such a copy with {func}`.get_particle_copy_by_name` and add it back with {func}`.add_to_particle_list`. This can for instance come in handy if you want to work with several $e^+e^-$ collision energies:

In [ ]:
from pycompwa.expertsystem.state import particle

In [ ]:
energies = [4180, 4220, 4420, 4600]  # MeV

for counter, energy in enumerate(energies, 1):
    new_particle = particle.get_particle_copy_by_name("EpEm")
    new_particle["Parameter"]["Value"] = energy / 1e3  # GeV
    pid = int(new_particle["Pid"]) + counter
    new_particle["@Name"] += str(energy)  # new name to not overwrite the old
    new_particle["Pid"] = str(pid)  # set unique PID
    particle.add_to_particle_list(new_particle)

assert original_number_of_entries + len(energies) == len(particle_list)

In [ ]:
from IPython.display import display

display(particle.particle_list["EpEm"]["Parameter"])
display(particle.particle_list["EpEm4180"]["Parameter"])

In [ ]:
particle.particle_list["D*(2010)+"]["DecayInfo"]["Parameter"][0][
    "Value"
] = 0.01

Finally, you can write particle to another XML file, so that you can have this particle list manipulation in a separate Python script:

In [ ]:
particle.write_particle_list_to_xml("new_particle_list.xml")

### (2) through XML

Alternatively, you can copy the default XML file and manipulate it by hand (copy a section, tweak some names, change some parameters) and load it as follows:

In [ ]:
particle.load_particle_list_from_xml("../particle_list.xml")

Note that duplicate values in the particle list are overwritten, but that old values are kept:

In [ ]:
print("Number of entries in particle_list")
print("  old:", original_number_of_entries)
print("  new:", len(particle.particle_list))

In [ ]:
particle_list["D*(2010)+"]["DecayInfo"]["Parameter"][0]["Value"]

In [ ]:
particle_list["EpEm4420"]["Parameter"]

## ComPWA particle list

ComPWA, which we manage through the {mod}`pycompwa.ui` module, works with a different particle list than the one discussed in the {ref}`previous section <usage/tools/particles:pycompwa particle list>`, because ComPWA's objects 'live' in the C++ backend. We saw this {class}`.ParticleList` class before, for instance when, {doc}`generating data </usage/workflow/step2>`. A {class}`pycompwa.ui.ParticleList` can be created from an XML file as follows:

In [ ]:
import pycompwa.ui

particles = pycompwa.ui.read_particles("../particle_list.xml")

You can append particles to this instance from another list with

In [ ]:
pycompwa.ui.Logging("error")  # switch off overwrite warnings
pycompwa.ui.insert_particles(particles, "../particle_list.xml")